In [1]:
from pairwise_model import *
from text_utils import *
import regex as re
from bm25_utils import BM25Gensim
from qa_model import *
from tqdm.auto import tqdm
tqdm.pandas()

c:\Users\Duc\miniconda3\envs\network\lib\site-packages\transformers\models\auto\tokenization_auto.py:823: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/405 [00:00<?, ?B/s]

c:\Users\Duc\miniconda3\envs\network\lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Duc\.cache\huggingface\hub\models--colbert-ir--colbertv2.0. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Note: to be able to use all crisp methods, you need to install some additional packages:  {'leidenalg', 'infomap', 'wurlitzer', 'graph_tool', 'bayanpy'}
Note: to be able to use all crisp methods, you need to install some additional packages:  {'pyclustering', 'ASLPAw'}
Note: to be able to use all crisp methods, you need to install some additional packages:  {'wurlitzer', 'infomap', 'leidenalg'}


In [2]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\Duc\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [3]:
df_wiki_windows = pd.read_parquet("E:\Smart Cloud\E2EQA\wikipedia_20220301.parquet")

In [ ]:
bm25_model_stage1 = BM25Gensim(r"E:\Smart Cloud\E2EQA\bm25\storage")

In [5]:
qa_model = QAEnsembleModel("bert-base-uncased", [r"E:\Smart Cloud\E2EQA\generation\pytorch_model.bin"])
pairwise_model_stage1 = PairwiseModel("colbert-ir/colbertv2.0").half()
pairwise_model_stage1.load_state_dict(torch.load(r"E:\Smart Cloud\E2EQA\pairwisemodel\pairwise_stackexchange.bin"))
pairwise_model_stage1.eval()

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cuda:0


PairwiseModel(
  (model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwis

In [8]:
question = "who is messi"

In [18]:
#Bm25 retrieval for top200 candidates
query = preprocess(question).lower()
top_n, bm25_scores = bm25_model_stage1.get_topk_stage1(query, topk=200)
texts = [preprocess(df_wiki_windows.text.values[i]) for i in top_n]

#Reranking with pairwise model for top100
question = preprocess(question)

print(question)
print(texts)

who is messi
['The music of the Cameroon includes diverse traditional and modern musical genres . The best-known contemporary genre is makossa , a popular style that has gained fans across Africa , and its related dance craze bikutsi . The pirogue sailors of Douala are known for a kind of singing called ngoso which has evolved into a kind of modern music accompanied by zanza , balafon , and various percussion instruments . Traditional music The ethnicities of Cameroon include an estimated 250 distinct ethnic groups in five regional-cultural divisions . An estimated 38 % of the population are Western highlanders–Semi-Bantu or grassfielders including the Bamileke , Bamum , and many smaller Tikar groups in the northwest . 12 % are coastal tropical forest peoples , including the Bassa , Duala , and many smaller groups in the southwest . The southern tropical forest peoples ( 18 % ) include the Beti-Pahuin and their sub-groups the Bulu and Fang , the Maka and Njem , as well as , the Baka py

In [11]:
ranking_preds = pairwise_model_stage1.stage1_ranking(question, texts)
ranking_scores = ranking_preds * bm25_scores

In [13]:
#Question answering
best_idxs = np.argsort(ranking_scores)[-10:]
ranking_scores = np.array(ranking_scores)[best_idxs]
texts = np.array(texts)[best_idxs]
best_answer = qa_model(question, texts, ranking_scores)
if best_answer is None: print("No answers")

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


No answers
